# rq & bs4 套件實作筆記[初階]
> by AHSNCCU 王修佑<br>
hugo.tw.wang@gmail.com

* 有興趣可以看看這些更詳細的官方文件<br>
> + [pandas-PyPI](https://pypi.org/project/pandas/)<br>
> + [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#beautiful-soup-documentation)<br>
> + [requests-PyPI](https://pypi.org/project/requests/)<br>

## 開始之前😉

如果你的環境沒有install這些套件要先`!pip install`窩！

In [ ]:
!pip install requests
!pip install pandas
!pip install BeautifulSoup

如果在Google Colab使用，要先架接上Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 先import套件

In [ ]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup

## 小試身手：刷校網點閱率

In [ ]:
url = 'https://www.ahs.nccu.edu.tw/ischool/public/news_view/show.php?nid=9968'

times = int(input('我要新增點閱次數?次'))

for i in range(times):
    rq.get(url)
    if i == (times-1)
        print('Done!')

完成後到校網看看你的點擊數成果吧！

---

## 開始實作吧！爬取政附校網公告文本

In [ ]:
ID = str(input('校網公告nid：'))
url = 'https://www.ahs.nccu.edu.tw/ischool/public/news_view/show.php?nid=' + ID

html = rq.get(url)
sp = soup(html.text, 'html.parser')

### select html tag
透過bs4剖析網頁，select目標位置<br>
> 可透過鍵盤`F12`查看Devoloper Tools

In [ ]:
title = sp.select('#title h4')
text = sp.select('#content p span')
info = sp.select('#info table tbody tr td span')

### 爬取資料
先存到暫存

In [ ]:
title_temp = ''
for t in title:
    title_temp += t.text

In [ ]:
department_temp = ''
staff_temp = ''
time_temp = ''
count = -1

for t in info:
    count += 1
    if count == (0 or 2 or 4):
        continue
    elif count == 1:
        department_temp = t.text
    elif count == 3:
        staff_temp = t.text
    elif count == 5:
        time_temp = t.text
    else:
        continue

In [ ]:
text_temp = ''
for t in text:
    text_temp += t.string

### DataFrame

建立一個2 rows * 6 columns的DataFrame，並加上columns name

In [ ]:
df = pd.DataFrame([[0,0,0,0,0,0],[0,0,0,0,0,0]], columns=['nid','title','department','staff','time','text'])
df

將剛才放在暫存的資料，放入DataFrame對應的位置

In [ ]:
df.loc[0,'nid'] = ID
df.loc[0,'title'] = title_temp
df.loc[0,'department'] = department_temp
df.loc[0,'staff'] = staff_temp
df.loc[0,'time'] = time_temp
df.loc[0,'text'] = text_temp

In [ ]:
df

### I/O
> 注意！繁體中文encoding請使用`utf_8_sig`<br>
to_csv官方文件：https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [ ]:
# df.to_csv('rq2.csv', header=None, index=None, encoding='utf_8_sig')
df.to_csv('ahsnccu_bulletin_CrawlPractice.csv', encoding='utf_8_sig')

---

## 完整實作-連續爬取大量資料

### 爬取

In [ ]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup

df = pd.DataFrame([[0,0,0,0,0,0]], columns=['nid','title','department','staff','time','text'])
df = df.drop(0)

for i in range(10000, 12200):
    ID = str(i)
    url = 'https://www.ahs.nccu.edu.tw/ischool/public/news_view/show.php?nid=' + ID
    html = rq.get(url)
    sp = soup(html.text, 'html.parser')
    
    title = sp.select('#title h4')
    text = sp.select('#content p span')
    info = sp.select('#info table tbody tr td span')
    
    title_temp = ''
    for t in title:
        title_temp += t.text
    
    department_temp = ''
    staff_temp = ''
    time_temp = ''
    count = -1

    for t in info:
        count += 1
        if count == (0 or 2 or 4):
            continue
        elif count == 1:
            department_temp = t.text
        elif count == 3:
            staff_temp = t.text
        elif count == 5:
            time_temp = t.text
        else:
            continue
    
    text_temp = ''
    for t in text:
        text_temp += t.text
        
    df = df.append(pd.DataFrame([[ID,title_temp,department_temp,staff_temp,time_temp,text_temp]],columns=['nid','title','department','staff','time','text']),ignore_index=True)

In [ ]:
df

### 處理空值

In [ ]:
# df.isnull().sum()
df1 = pd.DataFrame([[0,0,0,0,0,0]], columns=['nid','title','department','staff','time','text'])
df1 = df.drop(0)
df1 = pd.concat([df],ignore_index=True)

count = 0
for i in df1['title']:
    if i == '':
        df1 = df1.drop(count)
    count += 1

In [ ]:
df1

### I/O

In [ ]:
df1.to_csv('ahsnccu_bulletin_crawl.csv', encoding='utf_8_sig')